In [1]:
import time
import numpy as np
import pandas as pd

In [2]:
rd_schema = pd.read_json('../data/raw/pu2018_schema.json')
rd_schema['dtype'] = (['Int64' if x == 'integer'
                       else 'object' if x == 'string'
                       else 'Float64' if x == 'float'
                       else 'ERROR'
                       for x in rd_schema['dtype']]
                     )
rd_schema.dtype.value_counts()

Int64      4880
Float64      37
object       25
Name: dtype, dtype: int64

In [3]:
# Initialize lists with columns names to read from csv into df

# Standard columns suggested by SIPP documentation
base_cols = [#Common case-identification variables
    'SSUID','PNUM','MONTHCODE','ERESIDENCEID','ERELRPE','SPANEL','SWAVE',
    #The base weight and monthly in-survey-universe indicator
    'WPFINWGT','RIN_UNIV',
    #Common demographics variables, including age at time of interview (TAGE)
    #and monthly age during the reference period (TAGE_EHC)
    'ESEX','TAGE','TAGE_EHC','ERACE','EORIGIN','EEDUC',
    #Additional variables for analysis
    'TPTOTINC','RTANF_MNYN']

# Stock and 401k ownership columns
st_own_cols = ['EOWN_ST', 'EOWN_THR401']

# Concatenate columns to import
use_cols = base_cols + st_own_cols


In [4]:
# Load dataset. This works on my computer - takes about 66 seconds to read.
start = time.time()
df = pd.read_csv("../data/raw/pu2018.csv",
                      names=rd_schema['name'],
                      dtype=dict([(i,v) for i,v in zip(rd_schema.name, rd_schema.dtype)]),
                      sep='|',
                      header=0,
                      usecols=use_cols,
                     )
end = time.time()
print(f'Read with Pandas: {end - start} seconds')

Read with Pandas: 81.78944206237793 seconds


In [5]:
# Python friendly column name formatting
df.columns = [name.lower().replace(' ', '_') for name in df.columns]
df.head()

,ssuid,spanel,swave,pnum,erelrpe,esex,eorigin,erace,eeduc,eown_st,eown_thr401,monthcode,wpfinwgt,eresidenceid,rtanf_mnyn,rin_univ,tage,tage_ehc,tptotinc
0,00011413607018,2018,1,101,2,1,1,1,38,2,2,1,5971.747954,100002,2,1,33,32,1738
1,00011413607018,2018,1,101,2,1,1,1,38,2,2,2,5896.534769,100002,2,1,33,32,1738
2,00011413607018,2018,1,101,2,1,1,1,38,2,2,3,5845.211127,100002,2,1,33,32,1738
3,00011413607018,2018,1,101,2,1,1,1,38,2,2,4,5794.719891,100002,2,1,33,32,1738
4,00011413607018,2018,1,101,2,1,1,1,38,2,2,5,5790.817291,100002,2,1,33,32,1738


In [6]:
(df
 .groupby(['monthcode'])
 .eown_st
 .value_counts(normalize=True)
 .to_frame()
)

eown_st
monthcode eown_st          
1         2        0.888283
          1        0.111717
2         2        0.888283
          1        0.111717
3         2        0.888283
          1        0.111717
4         2        0.888283
          1        0.111717
5         2        0.888283
          1        0.111717
6         2        0.888283
          1        0.111717
7         2        0.888283
          1        0.111717
8         2        0.888283
          1        0.111717
9         2        0.888283
          1        0.111717
10        2        0.888283
          1        0.111717
11        2        0.888283
          1        0.111717
12        2        0.888283
          1        0.111717

In [7]:
df.describe()

,spanel,swave,pnum,erelrpe,esex,eorigin,erace,eeduc,eown_st,eown_thr401,monthcode,wpfinwgt,rtanf_mnyn,rin_univ,tage,tage_ehc,tptotinc
count,763186.0,763186.0,763186.000000,763186.000000,763186.000000,763186.000000,763186.000000,625896.000000,622464.000000,622464.000000,763186.000000,763186.000000,763186.000000,763186.000000,763186.000000,763186.000000,620018.000000
mean,2018.0,1.0,102.149105,4.253721,1.516031,1.811624,1.364357,40.333218,1.888283,1.712928,6.510540,5034.392912,1.993910,1.004958,40.394934,39.599683,3575.846667
std,0.0,0.0,1.358387,3.472320,0.499743,0.391012,0.775861,2.864025,0.315019,0.452396,3.452085,1589.148885,0.077802,0.070239,23.419359,23.428759,6822.266449
min,2018.0,1.0,101.000000,1.000000,1.000000,1.000000,1.000000,31.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,-126633.000000
25%,2018.0,1.0,101.000000,1.000000,1.000000,2.000000,1.000000,39.000000,2.000000,1.000000,4.000000,3826.676848,2.000000,1.000000,20.000000,19.000000,722.000000
50%,2018.0,1.0,102.000000,3.000000,2.000000,2.000000,1.000000,41.000000,2.000000,2.000000,7.000000,4832.714481,2.000000,1.000000,40.000000,39.000000,2112.000000
75%,2018.0,1.0,103.000000,7.000000,2.000000,2.000000,1.000000,43.000000,2.000000,2.000000,10.000000,6040.977440,2.000000,1.000000,60.000000,59.000000,4336.000000
max,2018.0,1.0,114.000000,18.000000,2.000000,2.000000,4.000000,46.000000,2.000000,2.000000,12.000000,24164.915109,2.000000,2.000000,87.000000,87.000000,518825.000000


In [9]:
# Create data dictionary
sipp_dict_1 = pd.read_csv('../data/raw/sippdict_1_of_2.csv')
sipp_dict_2 = pd.read_csv('../data/raw/sippdict_2_of_2.csv')
sipp_dict = pd.concat([sipp_dict_1, sipp_dict_2])
sipp_dict.columns = [name.lower().replace(' ', '_') for name in sipp_dict.columns]
sipp_dict.head()

,variable,topic,subtopic,survey_years,response_code,description,question,data_type,universe,universe_description,user_notes,record_level
0,EAWBCRACK,Adult and Child Well Being,Adult Well-Being,2018,1. Yes||2. No,Are there cracks in the ceiling or walls?,Are there cracks in the ceiling or walls?,Numeric,"THHLDSTATUS in (1,2,3,4)",All interviewed households (asked of reference...,"Descriptions, response codes, and universes ma...",Household
1,EAWBCRACK,Adult and Child Well Being,Adult Well-Being,"2014 Wave 4, 2014 Wave 3, 2014 Wave 2, 2014 Wa...",1. Yes||2. No,Are there cracks in the ceiling or walls?,Are there cracks in the ceiling or walls?,Numeric,All interviewed households.,All interviewed households (asked of reference...,"Descriptions, response codes, and universes ma...",Household
2,EAWBGAS,Adult and Child Well Being,Adult Well-Being,2018,1. Yes||2. No,Was ... unable to pay the utility bills?,Was ... unable to pay the utility bills?,Numeric,"THHLDSTATUS in (1,2,3,4)",All interviewed households (asked of reference...,"Descriptions, response codes, and universes ma...",Household
3,EAWBGAS,Adult and Child Well Being,Adult Well-Being,"2014 Wave 4, 2014 Wave 3, 2014 Wave 2, 2014 Wa...",1. Yes||2. No,Was ... unable to pay the utility bills?,Was ... unable to pay the utility bills?,Numeric,All interviewed households.,All interviewed households (asked of reference...,"Descriptions, response codes, and universes ma...",Household
4,EAWBHOLES,Adult and Child Well Being,Adult Well-Being,2018,1. Yes||2. No,Are there holes in the floor?,Are there holes in the floor?,Numeric,"THHLDSTATUS in (1,2,3,4)",All interviewed households (asked of reference...,"Descriptions, response codes, and universes ma...",Household


In [37]:
# Create boolean masks to drill down to dictionary variables
# Survey year filter
mask_2018 = sipp_dict.survey_years.str.contains('2018')

# Topic filter
list_topic = sipp_dict.topic.unique()        # Print this variable to view list of all topics
mask_topic = sipp_dict.topic == 'Demographics'

# Subtopic filter
list_subtopic = sipp_dict.subtopic.unique()   # Print this variable to view list of all subtopics
mask_subtopic = sipp_dict.subtopic == 'Age'

# Combine filters
mask_final = mask_2018 & mask_subtopic


sipp_dict[mask_final][['variable', 'description', 'response_code', 'data_type']]

,variable,description,response_code,data_type
1709,EDOB_BMONTH,Month of birth,1. January||2. February||3. March||4. April||5...,Numeric
1711,TAGE,Age as of last birthday,0:87,Numeric
1713,TAGE_EHC,Monthly age during the reference year.,0:87,Numeric
1715,TDOB_BYEAR,Year of birth,1931:2017,Numeric
1717,AAGE,Status Flag for TAGE,"0. Not in universe||1. In universe, as reporte...",Numeric
1719,ADOB_BMONTH,Status Flag for EDOB_BMONTH,"0. Not in universe||1. In universe, as reporte...",Numeric
1721,ADOB_BYEAR,Status Flag for TDOB_BYEAR,"0. Not in universe||1. In universe, as reporte...",Numeric
